In [1]:
import gtsam
import numpy as np
from gtsam.gtsam import (Cal3_S2, DoglegOptimizer,
                         GenericProjectionFactorCal3_S2, NonlinearFactorGraph,
                         Point3, Pose3, Point2, PriorFactorPoint3, PriorFactorPose3,
                         Rot3, SimpleCamera, Values)

from utils import get_matches_and_e, load_image

In [2]:
def symbol(name: str, index: int) -> int:
    """ helper for creating a symbol without explicitly casting 'name' from str to int """
    return gtsam.symbol(ord(name), index)

In [3]:
def get_camera_calibration(fx, fy, s, cx, cy):
  return Cal3_S2(fx, fy, s, cx, cy)

In [4]:
# Define the camera observation noise model
measurement_noise = gtsam.noiseModel_Isotropic.Sigma(2, 1.0)  # one pixel in u and v

In [5]:
img1 = load_image('img56.jpg', path='../data/lettuce_home/set6/')
img2 = load_image('img58.jpg', path='../data/lettuce_home/set6/')
points_1, points_2, e_estimate, r, t = get_matches_and_e(img1, img2)

selected points with ratio:  68


In [6]:
print(e_estimate)
print(r)
t = +(t/t[0])*0.05
print(t)

[[ 0.02161564 -0.1086115  -0.1994536 ]
 [ 0.2627051  -0.05079752 -0.62249989]
 [ 0.36620042  0.59124258  0.07199701]]
[array([[ 0.93237012, -0.26533176,  0.24553006],
       [ 0.26541593,  0.96355595,  0.03338131],
       [-0.24543907,  0.03404385,  0.96881406]]), array([[ 0.52519455, -0.72216304,  0.45016799],
       [-0.70583895, -0.66517095, -0.24359595],
       [ 0.47535466, -0.18981084, -0.85907496]])]
[[ 0.05      ]
 [-0.01510818]
 [ 0.00788698]]


In [7]:
# Create a factor graph
graph = NonlinearFactorGraph()
K = get_camera_calibration(644, 644, 0, 213, 387)

In [8]:
# add all the image points to the factor graph
for (i, point) in enumerate(points_1):
  # wrap the point in a measurement
  #print('adding point for camera1')
  factor = GenericProjectionFactorCal3_S2(
            Point2(point), measurement_noise, symbol('x', 0), symbol('l', i), K)
  graph.push_back(factor)
  
for (i, point) in enumerate(points_2):
  #print('adding point for camera2')
  factor = GenericProjectionFactorCal3_S2(
            Point2(point), measurement_noise, symbol('x', 1), symbol('l', i), K)
  graph.push_back(factor)

In [9]:
# Add a prior on pose of camera 1.
# 0.3 rad std on roll,pitch,yaw and 0.1m on x,y,z
pose_noise = gtsam.noiseModel_Diagonal.Sigmas(np.array([0.3, 0.3, 0.3, 0.1, 0.1, 0.1]))
factor = PriorFactorPose3(symbol('x', 0), Pose3(Rot3.Rodrigues(0, 0, 0), Point3(0, 0, 0)), pose_noise)
graph.push_back(factor)

In [10]:
# Add a prior on pose of camera 2
# 0.3 rad std on roll,pitch,yaw and 0.1m on x,y,z
pose_noise = gtsam.noiseModel_Diagonal.Sigmas(np.array([0.3, 0.3, 0.3, 0.1, 0.1, 0.1]))
factor = PriorFactorPose3(symbol('x', 1), Pose3(Rot3(r), Point3(t[0], t[1], t[2])), pose_noise)
graph.push_back(factor)

In [11]:
# point_noise = gtsam.noiseModel_Isotropic.Sigma(3, 0.1)
# factor = PriorFactorPoint3(symbol('l', 0), Point3(1,0,0), point_noise)
# graph.push_back(factor)

In [12]:
graph.print_('Factor Graph:\n')

In [13]:
# Create the data structure to hold the initial estimate to the solution
initial_estimate = Values()

r_init = Rot3.Rodrigues(0, 0, 0)
t_init = Point3(0, 0, 0)
transformed_pose = Pose3(r_init, t_init)
initial_estimate.insert(symbol('x', 0), transformed_pose)

r_init = Rot3(r)
t_init = Point3(t[0], t[1], t[2])
transformed_pose = Pose3(r_init, t_init)
initial_estimate.insert(symbol('x', 1), transformed_pose)

for j, point in enumerate(points_1):
  initial_estimate.insert(symbol('l', j), Point3(0.05*point[0]/640, 0.05*point[1]/640,0.05))
  

initial_estimate.print_('Initial Estimates:\n')

In [14]:
# Optimize the graph and print results
params = gtsam.DoglegParams()
params.setVerbosity('VALUES')
optimizer = DoglegOptimizer(graph, initial_estimate, params)
print('Optimizing:')

Optimizing:


In [15]:
result = optimizer.optimize()
result.print_('Final results:\n')
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

initial error = 48836395.56021277
final error = 5.3863958532626866
